In [1]:
from significance_of_mean_cuda import significance_of_mean_cuda
from utils import getdf
import numpy as np
import time
import multiprocessing
import concurrent.futures as cf
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
from IPython.display import Image
import sklearn
import time
from scipy.stats import ttest_ind, ttest_rel, chisquare, ks_2samp
import pandas as pd
import seaborn as sns

import matplotlib as mpl

mpl.rcParams['text.usetex'] = False  # not really needed

from scipy import stats
from scipy.stats import mannwhitneyu

### Calibration-plot

In [ ]:
def sns_plot(df, save_name=None):
    #sns.set(font_scale=1.5)
    sns.set(style="white")
    sns.set_context("talk")
    c1, c2 = df.columns
    low = min(min(df[c1]), min(df[c2]))
    hi = max(max(df[c1]), max(df[c2]))
    
    g=sns.lmplot(x=c1, y =c2, data=df, 
                  fit_reg=False, height=7, truncate=True, scatter_kws={"s": 15})
    
    g.set(xscale="log", yscale="log")
    axes = g.axes
    g.set(ylim=(low,hi), xlim=(low,hi))
    plt.plot([low,hi],[low,hi], "k", linewidth=1)
    sns.set_style("ticks")
    sns.despine()
    g.fig.tight_layout()
    if save_name:
        g.savefig(save_name)

In [ ]:
def multiple_plot(df, save_name):
    #sns.set(font_scale=1.5)
    sns.set(style="white")
    sns.set_context("talk")
    low = min(df["Theoretical p-value"])
    hi = max(df["Theoretical p-value"])
    g=sns.lmplot(x='Theoretical p-value', y ='Observed p-value', data=df, 
                  fit_reg=False, height=7, truncate=True, scatter_kws={"s": 15}, hue="Test")
    
    g.set(xscale="log", yscale="log")
    axes = g.axes
    g.set(ylim=(low,hi), xlim=(low,hi))
    plt.plot([low,hi],[low,hi], "k", linewidth=1)
    sns.set_style("ticks")
    sns.despine()
    g.fig.tight_layout()
    g.savefig(save_name)

In [ ]:
def exactTest(A,B, bins=10, one_side=True):
    SGM = significance_of_mean_cuda(bins, dtype_v=np.uint32,dtype_A=np.float64)
    SGM.run(np.asarray(A),np.asarray(B))
    if one_side:
        return SGM.get_p_values()
    else:
        return [ 2*p for p in SGM.get_p_values()]
        #return [2 * min( p, (1-p)) for p in SGM.get_p_values()]

def MWU(A, B, one_side=True):
    p_mw = list()
    for a,b in zip(A, B):
        if one_side:
            p_mw.append(mannwhitneyu(a,b, alternative="less")[1])
        else:
            p_mw.append(mannwhitneyu(a,b, alternative="two-sided")[1])
    return p_mw

def ttests(A,B, one_side=True):
    p_t = list()
    for x, y in zip(A, B):
        t, p = ttest_ind(y, x)
        if one_side:
            p = p/2
            if t<0:
                p = 1-p
        p_t.append(p)
    return p_t


In [ ]:
def getPATH(path, suffix, prefix):
    return path + '/'+ suffix + '/' + prefix

In [ ]:
path = "figures/paralellVsFastApprox"

# Set size 20 and sample size 10000

In [ ]:
import numpy as np
from significance_of_mean_cuda import significance_of_mean_cuda
from scipy.stats import mannwhitneyu, ttest_ind
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils import getdf

In [ ]:
np.random.seed(10)

In [ ]:
set_n = 20
sample_n = 20_000

# $N(0,1)$

In [ ]:

AN, BN = [np.random.normal(0,1,set_n) for i in range(sample_n)], [np.random.normal(0,1,set_n) for i in range(sample_n)]


# Exact test

In [ ]:
pExactListNorm = exactTest(BN,AN, 50, True)
#pDfExact = getdf(pExactListNorm, sample_n, "parallelized shift")
#my_scatter_plot(pDfExact,"figures/calibration/largeSample_10000_20/e_norm_0_1_50")

# Mann-Whitney

In [ ]:
pMWUListNorm = MWU(AN, BN, True)
#pDfMW = getdf(pMWUListNorm, sample_n, "MWU")
#my_scatter_plot(pDfMW,"figures/calibration/largeSample_10000_20/mw_norm_0_1_50")

# ttest

In [ ]:
pTtestListNorm = ttests(AN, BN, True)
#pDfTtest = getdf(pTtestListNorm, sample_n, "t test")
#my_scatter_plot(pDfTtest,"figures/calibration/largeSample_10000_20/t_norm_0_1_50")

In [ ]:
#multiple_plot(pd.concat((pDfTtest, pDfMW, pDfExact)), "figures/calibration/largeSample_10000_20/allTests_0_1_50")

In [ ]:
dataExactTtestNorm = pd.DataFrame(data={'parallelized': pExactListNorm, 'ttest': pTtestListNorm})
dataMWUTtestNorm = pd.DataFrame(data={'MWU': pMWUListNorm, 'ttest': pTtestListNorm})
dataEaxactMWUNorm = pd.DataFrame(data={'parallelized': pExactListNorm, 'MWU': pMWUListNorm})

In [ ]:
sns_plot(dataExactTtestNorm)

In [ ]:
sns_plot(dataEaxactMWUNorm)

In [ ]:
sns_plot(dataMWUTtestNorm)

# $log N(0,2)$

In [ ]:
std = 2

In [ ]:
ALn, BLn = [np.random.lognormal(0,std,set_n) for i in range(sample_n)], [np.random.lognormal(0,std,set_n) for i in range(sample_n)]


# Exact test

In [ ]:
pExactListLog = exactTest(BLn,ALn, 50, True)
#pDfExact = getdf(pExactListLog, sample_n, "parallelized shift")
#my_scatter_plot(pDfExact,"figures/calibration/largeSample_10000_20/e_Lnorm_0_1_50")

# Mann Whitney

In [ ]:
pMWUListLog = MWU(ALn, BLn, True)
#pDfMW = getdf(pMWUListLog, sample_n, "MWU")
#my_scatter_plot(pDfMW,"figures/calibration/largeSample_10000_20/mw_Lnorm_0_1_50")

# ttest

In [ ]:
pTtestListLog = ttests(ALn, BLn, True)
#pDfTtest = getdf(pTtestListLog, sample_n, "t test")
#my_scatter_plot(pDfTtest,"figures/calibration/largeSample_10000_20/t_Lnorm_0_1_50")

In [ ]:
#multiple_plot(pd.concat((pDfTtest, pDfMW, pDfExact)), "figures/calibration/largeSample_10000_20/allTests_Lnorm_0_1_50")

In [ ]:
dataExactTtestLog = pd.DataFrame(data={'parallelized': pExactListLog, 'ttest': pTtestListLog})
dataMWUTtestLog = pd.DataFrame(data={'MWU': pMWUListLog, 'ttest': pTtestListLog})
dataEaxactMWULog = pd.DataFrame(data={'parallelized': pExactListLog, 'MWU': pMWUListLog})

In [ ]:
sns_plot(dataExactTtestLog)

In [ ]:
sns_plot(dataEaxactMWULog)

In [ ]:
sns_plot(dataMWUTtestLog)